In [1]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.ml.feature import HashingTF, Normalizer
from pyspark.sql.types import StringType, ArrayType, DoubleType, IntegerType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
94,application_1575505342345_0139,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


### Rutas de Parquets files ha validar

In [2]:
####Paramters
str_matrizcampana = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_dmatrizcampana/'
str_fvtaproebecam = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_fvtaproebecam/'
str_tipooferta = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional-offline/dwh_dtipooferta/'
str_offert = 's3://belc-bigdata-landing-dlk-qas/lan-ganamas/input/cons/'
str_tiempocampana = 's3://belc-bigdata-functional-dlk-qas/analitico/GlueDatabase/functional/dwh_dtiempocampana/'

### Trae Información

In [3]:
df_tiempocampana = spark.read.parquet(str_tiempocampana)
df_dmatrizcampana = spark.read.parquet(str_matrizcampana)

In [4]:
ANIOCAMPAN_INI = '201901'
ANIOCAMPAN_FIN = '202004'

In [5]:
df_campana = df_tiempocampana.filter( 
    (col('codpais') != 'US') & 
    (col('aniocampana') >= ANIOCAMPAN_INI) & (col('aniocampana') <= ANIOCAMPAN_FIN)
).select('codpais', 'aniocampana')

## DMatrizCampana

Valida información:
* Se buscan ofertas digitales ( codtipooferta>'200'), que esten aspeados (codventa != '00000')
* PartitionBy (max_total) para buscar el total tipos de ofertas por campaña

In [6]:
#Condiciones
df_matriz1 = df_dmatrizcampana.filter( 
            (col('codtipooferta') >= '200') & (col('codventa') != '00000') & 
            (col('codpais') != 'US') & 
            (col('aniocampana') >= ANIOCAMPAN_INI) & (col('aniocampana') <= ANIOCAMPAN_FIN) 
        ).groupBy('codpais','aniocampana','codtipooferta')\
        .count().withColumnRenamed('count','total')\
        .sort(['codpais', 'aniocampana'])

#MaxTotal PartitionBy
window = Window.partitionBy('codpais','aniocampana' )

df_matriz = df_campana.join(df_matriz1,['codpais', 'aniocampana'],'left')\
        .select('codpais', 'aniocampana', 'codtipooferta','total').na.fill({'total': 0})\
        .withColumn("max_total",count('total').over(window))

df_matriz.filter( 
    (col('max_total') <= 3)
).orderBy(['aniocampana', 'codpais']).show(100)

+-------+-----------+-------------+-----+---------+
|codpais|aniocampana|codtipooferta|total|max_total|
+-------+-----------+-------------+-----+---------+
|     PR|     201912|         null|    0|        1|
|     MX|     201917|         null|    0|        1|
|     EC|     202003|          214|  282|        1|
|     BO|     202004|         null|    0|        1|
|     CL|     202004|         null|    0|        1|
|     CR|     202004|         null|    0|        1|
|     DO|     202004|         null|    0|        1|
|     EC|     202004|         null|    0|        1|
|     GT|     202004|         null|    0|        1|
|     MX|     202004|         null|    0|        1|
|     PA|     202004|         null|    0|        1|
|     PE|     202004|          211|    1|        2|
|     PE|     202004|          213|   51|        2|
|     PR|     202004|         null|    0|        1|
|     SV|     202004|         null|    0|        1|
+-------+-----------+-------------+-----+---------+